In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from model import VFAE, train
from dataset import DictionaryDataset
from checkpoint import Checkpoint
from data_generator import DataGen
from loss import VFAE_loss
from torch.utils.data import DataLoader

### VAE training

In [2]:
df = pd.read_csv('data/SF_cleaned.csv')
target = 'arrest_made'
sensitive = 'subject_race_black'
features = [col for col in df.columns.tolist() if col not in [target, sensitive]]
x, s, y = df[features], df[sensitive], df[target]

Xtrain, Xtest, strain, stest, ytrain, ytest = train_test_split(x,s,y,test_size=0.1)

In [3]:
train_data = {
    'x': np.array(Xtrain, dtype=np.float32),
    's': np.reshape(np.array(strain, dtype=np.float32), (-1,1),), 
    'y': np.reshape(np.array(ytrain, dtype=np.float32), (-1,1)) 
}

test_data = {
    'x': np.array(Xtest, dtype=np.float32),
    's': np.reshape(np.array(stest, dtype=np.float32), (-1,1),), 
    'y': np.reshape(np.array(ytest, dtype=np.float32), (-1,1)) 
}

In [4]:
# Hyperparams
x_dim = 70
s_dim = 1
y_dim = 1
z1_enc_dim = 350
z2_enc_dim = 350
z1_dec_dim = 350
x_dec_dim = 350
z_dim = 100
dropout_rate = 0.0

batch_size = 100000

alpha = 1
beta = 0.5
gamma = 1
mmd_dims = 500

In [5]:
vfae = VFAE(x_dim, s_dim, y_dim, z1_enc_dim, z2_enc_dim, z1_dec_dim, x_dec_dim, z_dim, dropout_rate, )
optim = torch.optim.Adam(vfae.parameters(),lr=1e-3)

In [6]:
dataset = DictionaryDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
loss_function = VFAE_loss(alpha=alpha, beta=beta, gamma=gamma, dims_out=mmd_dims)
checkpointer = Checkpoint('checkpoints')

In [ ]:
vfae.train()
for e in range(100):
    loss = train(e, vfae, train_dataloader, loss_function, optim, print_freq=250000)
    checkpointer(loss, vfae, e)

Train Epoch: 0 [0/814563 (0%)]	Loss: 53.227474
Train Epoch: 0 Average loss: 30.0869
Train Epoch: 1 [0/814563 (0%)]	Loss: 4.451664


In [5]:
vfae = VFAE(x_dim, s_dim, y_dim, z1_enc_dim, z2_enc_dim, z1_dec_dim, x_dec_dim, z_dim, dropout_rate, )
generator = DataGen(vfae, 'checkpoints/epoch-9-2.113356590270996.pth',)

In [6]:
test_dataset = DictionaryDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataloader), shuffle=False)
generator(test_dataloader)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0, 0.0, 0.0, 0.0)

#### Prediction with a classifier head from encoded x

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression

In [16]:
X_train, X_test, y_train, y_test = train_test_split(output['x_pred'],ytest,test_size=0.2)

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average = 'weighted')

NameError: name 'ypred' is not defined